In [86]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [87]:
!pip install opencv-python
!pip install opencv-contrib-python
!pip install matplotlib
!pip install pytesseract
!sudo apt install tesseract-ocr

Reading package lists... Done
Building dependency tree       
Reading state information... Done
tesseract-ocr is already the newest version (4.00~git2288-10f4998a-2).
0 upgraded, 0 newly installed, 0 to remove and 14 not upgraded.


In [88]:
import cv2
import random
import os
import glob 
import imutils
import pytesseract
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
pytesseract.pytesseract.tesseract_cmd = (r'/usr/bin/tesseract')

In [89]:
print(cv2.__version__)

4.1.2


In [90]:
ls -l '/content/gdrive/My Drive/Colab Notebooks/working_sample/data/images'

total 17927
-rw------- 1 root root   57064 Nov 28 06:10 3JOH22A.jpg
-rw------- 1 root root  468225 Dec  6 07:15 ALR486.png
-rw------- 1 root root   73112 Nov 28 06:10 BG224NZ.jpg
-rw------- 1 root root  581872 Dec  6 07:15 CZ17KOD.png
-rw------- 1 root root  266315 May 31  2020 DAN54P.png
-rw------- 1 root root  499510 Dec  6 07:23 DZI7YXR.png
-rw------- 1 root root  575888 May 31  2020 GBI8TCE.png
-rw------- 1 root root  501305 May 31  2020 GJ03JL0126.png
-rw------- 1 root root 1734822 Nov 28 06:10 GT7303V.jpg
-rw------- 1 root root  599460 May 31  2020 HR26BR9044.png
-rw------- 1 root root  629500 Dec  6 07:15 HR26BU0380.png
-rw------- 1 root root  562498 May 31  2020 HR26CU6799.png
-rw------- 1 root root 4291904 Nov 29 12:16 KA19MD7463.jpg
-rw------- 1 root root  181641 May 31  2020 KL01CA2555.png
-rw------- 1 root root  145785 May 31  2020 KL01CC50.png
-rw------- 1 root root 4598134 Nov 30 10:14 KL14U5815.jpg
-rw------- 1 root root  251866 May 31  2020 MH20EE7598.png
-rw------- 1 r

In [91]:
path_for_license_plates = "/content/gdrive/My Drive/Colab Notebooks/working_sample/data/images/*.*"
list_license_plates = [] 
predicted_license_plates = [] 
plateList = []

for path_to_license_plate in glob.glob(path_for_license_plates, recursive = True): 
  license_plate_file = path_to_license_plate.split("/")[-1] 
  license_plate, _ = os.path.splitext(license_plate_file) 
  print("Recognising:",license_plate)
  
  list_license_plates.append(license_plate) 

  img = cv2.imread(path_to_license_plate)

  gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  blur = cv2.bilateralFilter(gray, 11, 90, 90)
  edges = cv2.Canny(blur, 30, 200)
  cnts, new = cv2.findContours(edges.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
  cnts = sorted(cnts, key = cv2.contourArea, reverse = True)
  NumberPlateCount = None
  image_copy = img.copy()
  _ = cv2.drawContours(image_copy, cnts, -1, (255,0,255), 2)
  cnts = sorted(cnts, key=cv2.contourArea, reverse=True)[:30]
  image_copy = img.copy()
  _ = cv2.drawContours(image_copy, cnts, -1, (255,0,255), 2)

  plate = None
  for c in cnts:
      perimeter = cv2.arcLength(c, True)
      edges_count = cv2.approxPolyDP(c, 0.02 * perimeter, True)
      if len(edges_count) == 4:
        x,y,w,h = cv2.boundingRect(c)
        plate = img[y:y+h, x:x+w]
        predicted_result = pytesseract.image_to_string(plate, lang ='eng', config='--psm 10 --oem 3 -c tessedit_char_whitelist=ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789')
        filter_predicted_result = "".join(predicted_result.split())
        predicted_license_plates.append(filter_predicted_result) 
        break
      

Recognising: W0R516K
Recognising: MH20EE7598
Recognising: MK3532
Recognising: KL01CA2555
Recognising: HR26CU6799
Recognising: TN99F2378
Recognising: ALR486
Recognising: CZ17KOD
Recognising: HR26BU0380
Recognising: DZI7YXR
Recognising: BG224NZ
Recognising: YJ65YOM
Recognising: YJ65YKN
Recognising: 3JOH22A
Recognising: PGMN112
Recognising: GT7303V
Recognising: KA19MD7463
Recognising: KL14U5815
Recognising: GJ03JL0126
Recognising: GBI8TCE
Recognising: DAN54P
Recognising: HR26BR9044
Recognising: PUI8BES
Recognising: KL01CC50


In [92]:
print(list_license_plates)
print(len(list_license_plates))

['W0R516K', 'MH20EE7598', 'MK3532', 'KL01CA2555', 'HR26CU6799', 'TN99F2378', 'ALR486', 'CZ17KOD', 'HR26BU0380', 'DZI7YXR', 'BG224NZ', 'YJ65YOM', 'YJ65YKN', '3JOH22A', 'PGMN112', 'GT7303V', 'KA19MD7463', 'KL14U5815', 'GJ03JL0126', 'GBI8TCE', 'DAN54P', 'HR26BR9044', 'PUI8BES', 'KL01CC50']
24


In [93]:
print(predicted_license_plates)
print(len(predicted_license_plates))

['[iorsick', 'MH20EE7598', 'MK-35-32', 'KEG410A2555', 'HR26CU6799', 'TNSSF2378|', 'ALR486|', 'CZI7KOD', 'HR26BU380.', 'DZI7YXR', 'BG#224:NZ', 'YJ65YOM', 'r7J65YK', '3JOH22A', "PGeMN112'", '6T7303V', 'KA19.MD7463', 'L1405815', 'GJ03JL0126', 'GBI8TCE', 'DAN54P', "'HR26BR9044|", '|PUIGBES', 'KL01CC59']
24


In [98]:
print("Sl.No", "\t","Actual License Plate", "\t", "Predicted License Plate", "\t", "Accuracy") 
print("------", "\t","--------------------", "\t", "-----------------------", "\t", "--------") 

accuracyList = []
def calculate_predicted_accuracy(actual_list, predicted_list): 
  count = 0
  punctuations = '''!|()-[]{};:'"\,<>. /?@#$%^&*_~''' 
  for actual_plate, predicted_plate in zip(actual_list, predicted_list):
    for character in predicted_plate:
      if character in punctuations:
        predicted_plate = predicted_plate.replace(character, "")
    count = count + 1 
    accuracy = "0.0 %"
    num_matches = 0
    actual_plate = actual_plate.strip()
    predicted_plate = predicted_plate.strip()
    if actual_plate == predicted_plate: 
      accuracy = "100 %"
    else: 
      if len(actual_plate) == len(predicted_plate): 
        for a, p in zip(actual_plate, predicted_plate): 
          if a == p: 
            num_matches += 1
          accuracy = str(round((num_matches / len(actual_plate)), 2) * 100) 
          accuracy += "%"
    for i in accuracy:
      if i ==  '%':
        temp = accuracy.replace(i, "")
    accuracyList.append(float(temp))
    print(" ",str(count).zfill(2),"\t\t", actual_plate, "\t\t", predicted_plate, "\t\t ", accuracy) 


calculate_predicted_accuracy(list_license_plates, predicted_license_plates)
print("\nFinal Accuracy:",sum(accuracyList)/len(accuracyList))
accuracyList.clear()

 Sl.No 	 Actual License Plate 	 Predicted License Plate 	 Accuracy
------ 	 -------------------- 	 ----------------------- 	 --------
  01 		 W0R516K 		 iorsick 		  0.0%
  02 		 MH20EE7598 		 MH20EE7598 		  100 %
  03 		 MK3532 		 MK3532 		  100 %
  04 		 KL01CA2555 		 KEG410A2555 		  0.0 %
  05 		 HR26CU6799 		 HR26CU6799 		  100 %
  06 		 TN99F2378 		 TNSSF2378 		  78.0%
  07 		 ALR486 		 ALR486 		  100 %
  08 		 CZ17KOD 		 CZI7KOD 		  86.0%
  09 		 HR26BU0380 		 HR26BU380 		  0.0 %
  10 		 DZI7YXR 		 DZI7YXR 		  100 %
  11 		 BG224NZ 		 BG224NZ 		  100 %
  12 		 YJ65YOM 		 YJ65YOM 		  100 %
  13 		 YJ65YKN 		 r7J65YK 		  0.0%
  14 		 3JOH22A 		 3JOH22A 		  100 %
  15 		 PGMN112 		 PGeMN112 		  0.0 %
  16 		 GT7303V 		 6T7303V 		  86.0%
  17 		 KA19MD7463 		 KA19MD7463 		  100 %
  18 		 KL14U5815 		 L1405815 		  0.0 %
  19 		 GJ03JL0126 		 GJ03JL0126 		  100 %
  20 		 GBI8TCE 		 GBI8TCE 		  100 %
  21 		 DAN54P 		 DAN54P 		  100 %
  22 		 HR26BR9044 		 HR26BR9044 		  100 %
  23 		 PU